# Twitter sentiment vs BTC USD 

## Import Dependencies

In [1]:
%matplotlib notebook

In [2]:
from config import (consumer_key, consumer_secret, access_token, access_token_secret)

In [3]:
import  matplotlib.pyplot  as plt
import pandas as pd
import numpy as np
import tweepy
import json
import datetime
import time
import pprint as pprint
import requests
import json
import sklearn.preprocessing as skl


from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

## Using mini-api for cryptocurrency for price changes

In [4]:
url = "https://min-api.cryptocompare.com/data/histominute?fsym=BTC&tsym=GBP&limit=95"
response = requests.get(url)
print(response.url)

https://min-api.cryptocompare.com/data/histominute?fsym=BTC&tsym=GBP&limit=95


## Create Twitter Authorization

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

## Create module to run for twitter sentiments

In [6]:
def twitter_search():
    
    public_tweets = api.search('BTCUSD', count=96, result_type="recent")
        
        # Loop through all tweets
    for tweet in public_tweets['statuses']:
    
           # Run Vader Analysis on each tweet
        results = analyzer.polarity_scores(tweet["text"])
        compound = results["compound"]
        pos = results["pos"]
        neu = results["neu"]
        neg = results["neg"]
    
            # Add each value to the appropriate list
        compound_list.append(compound)
        positive_list.append(pos)
        negative_list.append(neg)
        neutral_list.append(neu)
        
        sentiments.append({"Date": tweet["created_at"], 
                           "Compound": compound,
                           "Positive": pos,
                           "Negative": neu,
                           "Neutral": neg,
                           "Text": tweet['text'],
                           })
            
        time = tweet['created_at']
        convert_time(time)
            
    #plt.plot(np.arange(len(compound_list)), compound_list)
    #plt.title(f"Sentiment Analysis of Tweets")
    #plt.ylabel("Tweet Polarity")
    #plt.xlabel("Time stamp")
    #plt.xticks(rotation = 'vertical')
    #plt.tight_layout()
    #plt.savefig('../Output/Sentiment_Analysis')

## Create Module to convert time stamp

In [7]:
def convert_time (time):
    time = datetime.datetime.strptime(time, "%a %b %d %H:%M:%S %z %Y").timestamp()
    time = datetime.datetime.fromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')
    time_stamp.append(time)

## Create module for recent 7 day price change

In [8]:
def refresh_price():
    Timestamp = []
    Close = []
    converted_time = []
    #extract timestamp and USD price from json
    data = response.json()
    #pprint(data)
    for x in data['Data']:
        Timestamp.append(x['time'])
        Close.append(x['close'])
    
    #create data frame from file and then add a converted timestamp column to df
    for each in Timestamp:
        converted_time.append(str(each))
        
    price_by_minute = [('Timestamp', Timestamp),('Close',Close), ('Converted Time',converted_time)]
    btc_prices = pd.DataFrame.from_items(price_by_minute)
    btc_prices['Converted Time'] = pd.to_datetime(btc_prices['Timestamp'], unit = 's')
    btc_prices.set_index(['Converted Time'], inplace = True)
    
    #normalize prices
    btc_norm = skl.normalize(btc_prices['Close'].values.reshape(-1,1), axis = 0, norm = 'l1')

    #create plot
    plt.plot(np.arange(len(compound_list)), btc_norm)
    plt.plot(np.arange(len(compound_list)), compound_list)
    plt.xlabel('Date')
    plt.ylabel('USD')
    plt.xticks(rotation = 'vertical')
    plt.tight_layout()
    plt.show()
    plt.savefig('../Output/btc_mostrecentSeven.png')

In [9]:
compound_list = []
positive_list = []
neutral_list = []
negative_list = []
sentiments = []
time_stamp = []
    
twitter_search()
refresh_price

<function __main__.refresh_price>

In [14]:
sentiments_pd = pd.DataFrame(sentiments)
sentiments_pd.set_index(['Date'], inplace = True)
sentiments_pd.head()

,Compound,Negative,Neutral,Positive,Text
Date,,,,,
Tue Jul 10 03:50:42 +0000 2018,0.2023,0.924,0.000,0.076,RT @DecentraIizing: $BTCUSD weekly bounced off...
Tue Jul 10 03:46:27 +0000 2018,-0.1779,0.825,0.175,0.000,"So this is it? C'mon, seriously now... $BTCUSD..."
Tue Jul 10 03:45:05 +0000 2018,0.0000,1.000,0.000,0.000,BitMEX BTCUSD funding payout in 15 minutes: 0....
Tue Jul 10 03:43:37 +0000 2018,0.0772,0.874,0.000,0.126,"So this is it? ""I want to believe"" $BTCUSD htt..."
Tue Jul 10 03:43:03 +0000 2018,-0.3182,0.892,0.108,0.000,RT @SamuelPower39: This is why I stick with th...


In [15]:
btc_prices

NameError: name 'btc_prices' is not defined